In [16]:
from IPython.core.display import HTML
import pandas as pd
import numpy as np
import imp
import help_tools as ht
HTML("<style>.container { width: 120% !important; }</style>")

In [ ]:
api_key_tmdb = '' 
api_key_omdb = ''

In [20]:
# откуда накидывать id людей
# https://www.themoviedb.org/person/7030-henry-goodman

actor_dict = {
#             'joaquin-phoenix' : 73421,
#             'colin-farrell' : 72466,
            'leonardo-dicaprio' : 6193,
#             'javier-bardem' : 3810,
#             'anthony-hopkins' : 4173,
#             'brad-pitt' : 287,
#             'antonio-banderas': 3131,
#             'matthew-mcconaughey' : 10297,
#             'steve-buscemi' : 884,
#             'tim-roth' : 3129,
#             'mads-mikkelsen' : 1019,
#             'scarlett-johansson' : 1245,
#             'til-schweiger' : 1844
             }

director_dict = {
#                 'quentin-tarantino' : 138,
#                 'david-fincher' : 7467,
#                 'jim-jarmusch' : 4429,
                'guy-ritchie' : 956,
#                 'christopher-nolan' : 525,
#                 'darren-aronofsky' : 6431,
#                 'martin-mcdonagh' : 54472,
#                 'pedro-almodovar' : 309,
#                 'paolo-sorrentino' : 56194,
#                 'cristian-mungiu' : 20657,
#                 'yorgos-lanthimos' : 122423,
#                 'bong-joon-ho' : 21684,
#                 'lars-von-trier' : 42
}

# фильмы которые не хочется чекать (уже просмотренные или не интересные)
excl_films = [335977, 799583, 882569, 800158, 872585, 696506, 1029955]

# предварительные фильтры (по имеющимся параметрам в фильмографии - чтобы дальше не тратить запросы на детализацию)
vote_average_min = 5
vote_count_min = 100
release_date_min = '2022-01-01'
language = 'ru' # добавлять русскояз названия если есть в прокате 

In [21]:
# ПОЛУЧАЕМ ИНФУ ИЗ TMDB + OMDB (РАБОТА С API)
flt_dict = {'vote_average_min' : vote_average_min, 
            'vote_count_min' : vote_count_min,
            'release_date_min' : release_date_min,
            'language' : language
           }

# получаем инфу по списку актеров
df1 = ht.get_actors_filmo(api_key_tmdb, actor_dict, excl_films = excl_films, flt_dict = flt_dict)
# получаем инфу по списку режиссеров
df2 = ht.get_directors_filmo(api_key_tmdb, director_dict, excl_films = np.append(df1.movie_id.values, excl_films), flt_dict = flt_dict)
# итого
df = pd.concat([df1, df2], ignore_index = True)
df_final = ht.get_imdb_info(api_key_omdb, df)

In [26]:
# ПОСТ ФИЛЬТРАЦИЯ И СОРТИРОВКИ ИЗ ДОП УСЛОВИЙ (НЕТ РАБОТЫ С API)
post_excl_films = []
get_filter(df_final, 
           vote_count_min=100, # vote cnt on tmdb
           runtime_min = 70, # film duration
           vote_average_min = 5, # rating по tmdb
           imdb_rating_min = 6, 
           release_date_min = '2022-01-01',
           genres_flt = ['Animation', 'Documentary'], # жанры которые неинтересно отображать
           excl_films = np.append(excl_films, post_excl_films), # фильмы которые неинтересно отображать
           drop_columns=['movie_status', 'movie_id', 'countries'] # какие колонки скрыть
          ).sort_values(by='imdb_rating', ascending=False)

,person,person_type,title,original_title,directors,release_date,imdb_rating,awards,genres,runtime,imdb_id,vote_average,vote_count
0,leonardo-dicaprio,actor,Убийцы цветочной луны,Killers of the Flower Moon,Martin Scorsese,2023-10-18,7.6,Nominated for 10 Oscars. 136 wins & 427 nomina...,"['Crime', 'History', 'Drama']",206,tt5537002,7.430,3576
1,guy-ritchie,director,Операция «Фортуна»: Искусство побеждать,Operation Fortune: Ruse de Guerre,,2023-01-04,6.3,N/A,"['Action', 'Thriller', 'Comedy']",113,tt7985704,6.485,1648


### Spotify

In [29]:
# get id + secret on your : https://developer.spotify.com/dashboard/e7bd2deea29b4fe1a33392499c720f45
client_id = ''
client_secret = ''

In [97]:
# посмотреть айдишки наиболее популярных исполнителей по имени
artist_name = 'mans'
top_cnt = 5
ht.get_top_n_id_by_name(client_id, client_secret, artist_name, top_cnt)

['Marilyn Manson - 2VYQTNDsvvKN9wmU5W7xpj - popularity: 72',
 "Poor Man's Poison - 0YHgnSkV3S5mvSSCTRWDi5 - popularity: 66",
 'Britney Manson - 7GtPtPbDC5CLNzSNxVpqaf - popularity: 61',
 'Mansur - 1nXS8JvKsTNSGw75Axv6rm - popularity: 66',
 'Mansheel Gujral - 7zfceKY7I04Tc0J2o7043h - popularity: 63']

In [59]:
artist_list = [
#                 '132sZpCaM8ie6byAEcOcRs', # haelos
               '2VYQTNDsvvKN9wmU5W7xpj', # manson
#               '3w4VAlllkAWI6m0AV0Gn6a', # hurts,
#                '4njdEjTnLfcGImKZu1iSrz', # awolnation
#                '3Bd1cgCjtCI32PYvDC3ynO', # london gram
#                '3XHO7cRUPCLOr6jwp8vsx5', # alt j
#                '163tK9Wjr9P9DmM0AVK7lm', # lorde
#                '3iOvXCl6edW5Um0fXEBRXy', # xx
#               '0RqtSIYZmd4fiBKVFqyIqD', # 30 second
#                '5Pwc4xIPtQLFEnJriah9YJ', # one republ
#               '1WgXqy2Dd70QQOU7Ay074N', # aurora,
#                 '4Z8W4fKeB5YxbusRsdQVPb', # radiohead
#                '0BEI7i5sgUuivcfwXLzFmM', # serj t
#                '3wRt3iJpZDOg73CTUkfv5C', # svrcina
#                 '0YCqg8X5gx0MremW64Ud1h', # ддт
#                 '6mdiAmATAx73kdxrNrnlao', # iron m
#                 '2ye2Wgw4gimLv2eAKyk1NB', # metallica
#                 '1moxjboGR7GNWYIMWsRjgG', # florence + m
#                 '1AZ30JnvQU1pbX6sbRE0Yn', # poets of fall
#                 '2N0vFuOoMtAQfBmhsRo24e' # kalanda
]

df_spotify = ht.sp_get_albums_info(client_id, client_secret, artist_list)

In [63]:
# ПОСТ ФИЛЬТРАЦИЯ (АПИ НЕ РАБОТАЕТ ЗДЕСЬ)
ht.sp_get_filter(df_spotify, 
                release_date_min = '2010-01-01', 
                release_date_max = '2030-01-01',
                album_flt = ['deluxe', 'live', 'edition', 'soundtrack'], # исключаем слова в названии альбомов
                album_type_flt = ['single'], # убрать эти типы
                max_artists_in_album = 1,
                other_album_author = True, # false когда не хотим чтобы это был фит артиста в альбом другого чувака
                 columns_excl = ['uri', 'album_type'] # какие колонки не отображать здесь
                 )

,artist,album,release_date,tracks_cnt,album_artists
0,Marilyn Manson,One Assassination Under God - Chapter 1,2024-11-22,9,['Marilyn Manson']
1,Marilyn Manson,Hybrid Theory (20th Anniversary Edition),2020-10-09,80,['Linkin Park']
2,Marilyn Manson,Floor Seats II,2020-09-25,11,['A$AP Ferg']
3,Marilyn Manson,WE ARE CHAOS,2020-09-11,10,['Marilyn Manson']
4,Marilyn Manson,24 Hours To Live (Original Motion Picture Soun...,2017-12-29,20,['Tyler Bates']
5,Marilyn Manson,Heaven Upside Down,2017-10-06,10,['Marilyn Manson']
6,Marilyn Manson,Countach (For Giorgio),2016-03-11,9,['Shooter Jennings']
7,Marilyn Manson,The Pale Emperor,2015-01-19,13,['Marilyn Manson']
8,Marilyn Manson,Silent So Long,2014-11-07,11,['Emigrate']
9,Marilyn Manson,Avril Lavigne (Expanded Edition),2013-11-01,16,['Avril Lavigne']
